We want to sell our Sales AI software. However, not all companies want to buy them, and there are some that can't afford them. 

Thus, it is very important to idenfity the best customer base, so that most of our company's energy is spend on pitching to such customers, at least initially when we don't have any customers. 

The below clustering model will provide with the best set of companies that are most likely to purchase our product.

In [828]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns

Now, we will import data and perform some Exploratory Data Analysis (EDA)

In [829]:
inc_5000_data = pd.read_csv("/kaggle/input/inc-5000-companies/INC 5000 Companies 2019.csv")

In [830]:
# function to understand the dataset
def understand_dataset(dataset):
    print("Type = " +str(type(dataset))+"\n")
    print("Shape = "+str(dataset.shape)+"\n")
    print("Head : \n\n"+str(dataset.head())+"\n\n")
    print(str(dataset.info())+"\n\n")
    print("No.of unique values :\n\n"+str(dataset.nunique(axis=0))+"\n\n")
    print("Description :\n\n"+str(dataset.describe())+"\n\n")
    print("Unique Industries :\n\n"+str(dataset['industry'].unique())+"\n")
    print("Revenue Scale "+str(inc_5000_data['revenue'].apply(lambda x:x.split(' ')[1]).sort_values().unique())+"\n")
    #Since no categorical variables, no need to have the above line
    
    print("Null count :\n\n"+str(dataset.isnull().sum()))
    
understand_dataset(inc_5000_data)

Type = <class 'pandas.core.frame.DataFrame'>

Shape = (5012, 14)

Head : 

   rank                                   profile               name  \
0     1      https://www.inc.com/profile/freestar           Freestar   
1     2   https://www.inc.com/profile/freightwise        FreightWise   
2     3  https://www.inc.com/profile/ceces-veggie  Cece's Veggie Co.   
3     4      https://www.inc.com/profile/ladyboss           LadyBoss   
4     5        https://www.inc.com/profile/perpay             Perpay   

                         url state       revenue    growth_%  \
0        http://freestar.com    AZ  36.9 Million  36680.3882   
1  http://freightwisellc.com    TN  33.6 Million  30547.9317   
2   http://cecesveggieco.com    TX  24.9 Million  23880.4852   
3        http://ladyboss.com    NM  32.4 Million  21849.8925   
4          http://perpay.com    PA  22.5 Million  18166.4070   

                       industry  workers  previous_workers  founded  \
0       Advertising & Marketing     

Based on the above view of data, we decide to proceed as following :

1) We eliminate columns like profile, url, state, previous_workers, yrs_on_list, metro and city

2) We will make 'Name' as the index of this dataset

3) We will convert 'revenue' from text to numeric format

4) We will convert 'founded' to 'years_active' 

5) We will only filter for 1 industry i.e. 'software' for this exercise

In [831]:
inc_5000_data = inc_5000_data.drop(['profile', 'url', 'state', 'previous_workers', 'yrs_on_list', 'metro' , 'city'],axis=1)

In [832]:
inc_5000_data = inc_5000_data.set_index('name')

In [833]:
inc_5000_data['revenue']

name
Freestar                  36.9 Million
FreightWise               33.6 Million
Cece's Veggie Co.         24.9 Million
LadyBoss                  32.4 Million
Perpay                    22.5 Million
                              ...     
Village Plumbing & Air    15.8 Million
Real Restoration Group    11.6 Million
Naval Systems             29.7 Million
HNM Systems                8.8 Million
Vivayic                    4.5 Million
Name: revenue, Length: 5012, dtype: object

In [834]:
inc_5000_data['revenue'] = inc_5000_data['revenue'].apply(lambda x:float(x[:-8]) * pow(10,9) if 'Billion' in x else float(x[:-8]) * pow(10,6))

In [835]:
inc_5000_data['years_active'] = 2022 - inc_5000_data['founded']
inc_5000_data = inc_5000_data.drop(['founded'],axis=1)

In [836]:
inc_5000_data = inc_5000_data[inc_5000_data['industry'] == 'Software']
inc_5000_data = inc_5000_data.drop(['industry'],axis=1)

In [837]:
inc_5000_data

,rank,revenue,growth_%,workers,years_active
name,,,,,
ConvertKit,23,13800000.0,7965.9741,37.0,9
Printify,24,10500000.0,7926.4244,70.0,7
Awardco,25,8900000.0,7478.1478,42.0,7
LumenAd,29,11800000.0,7045.3631,60.0,8
Beeswax,46,16900000.0,5768.1833,56.0,8
...,...,...,...,...,...
Corevist,4961,3300000.0,53.2110,15.0,14
WorkForce Software,4964,100900000.0,53.1734,550.0,23
ClearlyRated,4969,4100000.0,52.9602,33.0,19


This is the data on which we will perform Clustering

## Clustering

In [838]:
# Standard scaling (Usually from 0 to 1) is mandatory for a clustering problem

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
inc_5000_data_cluster = pd.DataFrame(scaler.fit_transform(inc_5000_data),columns=inc_5000_data.columns,index=inc_5000_data.index)

In [839]:
inc_5000_data_cluster

,rank,revenue,growth_%,workers,years_active
name,,,,,
ConvertKit,0.000000,0.030499,1.000000,0.020612,0.057143
Printify,0.000201,0.021970,0.995002,0.040047,0.000000
Awardco,0.000403,0.017834,0.938355,0.023557,0.000000
LumenAd,0.001208,0.025330,0.883665,0.034158,0.028571
Beeswax,0.004630,0.038511,0.722272,0.031802,0.028571
...,...,...,...,...,...
Corevist,0.993961,0.003360,0.000087,0.007656,0.200000
WorkForce Software,0.994565,0.255622,0.000082,0.322733,0.457143
ClearlyRated,0.995572,0.005428,0.000055,0.018257,0.342857


In [840]:
# function to understand the dataset
def understand_dataset_cluster(dataset):
    print("Head : \n\n"+str(dataset.head())+"\n\n")
    print(str(dataset.info())+"\n\n")    
    print("Null count :\n\n"+str(dataset.isnull().sum()))
    
understand_dataset_cluster(inc_5000_data_cluster)

Head : 

                rank   revenue  growth_%   workers  years_active
name                                                            
ConvertKit  0.000000  0.030499  1.000000  0.020612      0.057143
Printify    0.000201  0.021970  0.995002  0.040047      0.000000
Awardco     0.000403  0.017834  0.938355  0.023557      0.000000
LumenAd     0.001208  0.025330  0.883665  0.034158      0.028571
Beeswax     0.004630  0.038511  0.722272  0.031802      0.028571


<class 'pandas.core.frame.DataFrame'>
Index: 461 entries, ConvertKit to Distillery
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   rank          461 non-null    float64
 1   revenue       461 non-null    float64
 2   growth_%      461 non-null    float64
 3   workers       461 non-null    float64
 4   years_active  461 non-null    float64
dtypes: float64(5)
memory usage: 21.6+ KB
None


Null count :

rank            0
revenue         0
growth_%        0
w

In [841]:
from sklearn.cluster import KMeans

# Create a KMeans model with 4 clusters
kmeans = KMeans(n_clusters=4,random_state=1)

# Use the fit_predict method to cluster the data
clusters = kmeans.fit_predict(inc_5000_data_cluster)

In [842]:
company_names = pd.Series(inc_5000_data_cluster.index)
company_names

0              ConvertKit
1                Printify
2                 Awardco
3                 LumenAd
4                 Beeswax
              ...        
456              Corevist
457    WorkForce Software
458          ClearlyRated
459        Film Solutions
460            Distillery
Name: name, Length: 461, dtype: object

In [843]:
cluster_no = pd.Series(clusters,name='Cluster')
cluster_no

0      2
1      2
2      2
3      2
4      2
      ..
456    3
457    3
458    3
459    3
460    3
Name: Cluster, Length: 461, dtype: int32

In [844]:
Company_cluster = pd.concat([inc_5000_data.reset_index(),cluster_no],axis=1)

In [845]:
Company_cluster

,name,rank,revenue,growth_%,workers,years_active,Cluster
0,ConvertKit,23,13800000.0,7965.9741,37.0,9,2
1,Printify,24,10500000.0,7926.4244,70.0,7,2
2,Awardco,25,8900000.0,7478.1478,42.0,7,2
3,LumenAd,29,11800000.0,7045.3631,60.0,8,2
4,Beeswax,46,16900000.0,5768.1833,56.0,8,2
...,...,...,...,...,...,...,...
456,Corevist,4961,3300000.0,53.2110,15.0,14,3
457,WorkForce Software,4964,100900000.0,53.1734,550.0,23,3
458,ClearlyRated,4969,4100000.0,52.9602,33.0,19,3
459,Film Solutions,4983,7400000.0,52.6491,47.0,16,3


In [846]:
def understand_dataset_cluster(dataset):
    print(str(dataset.describe())+"\n\n")    

for c in range(4):
    print("Cluster "+str(c))
    understand_dataset_cluster(Company_cluster[Company_cluster['Cluster'] == c].drop(['Cluster'],axis=1))
    print("\nCompany List - "+str(list(Company_cluster[Company_cluster['Cluster'] == c].name))+"\n")

Cluster 0
              rank       revenue    growth_%      workers  years_active
count    26.000000  2.600000e+01   26.000000    26.000000     26.000000
mean   2862.230769  1.299385e+08  162.988200   694.730769     18.692308
std    1050.272586  8.455230e+07  114.121889   357.817334      5.836226
min     706.000000  7.300000e+06   57.091200    43.000000     11.000000
25%    2222.000000  7.525000e+07   96.535625   502.750000     15.000000
50%    2722.500000  1.105500e+08  140.795250   625.000000     18.500000
75%    3614.000000  1.560000e+08  181.067450   894.250000     21.750000
max    4836.000000  3.889000e+08  619.234600  1700.000000     37.000000



Company List - ['Zoom Video Communications', 'Legwork Software', 'iTechArt Group', 'DiscoverOrg', 'Ivalua', 'Smarsh', 'MediaAlpha', 'AvidXchange', 'Modernizing Medicine', 'WebPT', 'Diligent Corporation', 'AgileThought', 'Trintech', 'Turbonomic', 'Kyriba', 'Chrome River', 'LLamasoft', 'iCIMS', 'Workfront', 'Gear', 'Bigcommerce', 'Coherent

The above data shows the companies which are clustered and what is their property

For e.g. Cluster 2 has much higher mean revenue growth %, but Cluster 0 has highest mean absolute revenue amount amongst all cluste

## As a result, companies in clusters 0 and 2 can be targeted over other clusters